In [1]:
import os

from langchain import hub
from langchain.agents import initialize_agent, Tool, AgentExecutor


from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from tavily import TavilyClient

import requests

load_dotenv() 


/Users/aditiphadnis/Documents/Github Projects/SDS-CP022-ai-travel-companion/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


False

In [2]:
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents import AgentType
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import requests

In [3]:
urls = {
    'top':'https://www.tripadvisor.com/TravelersChoice-Destinations-cTop-g1',
    'trending':'https://www.tripadvisor.com/TravelersChoice-Destinations-cTrending-g1',
    'food':'https://www.tripadvisor.com/TravelersChoice-Destinations-cFood-g1',
    'culture':'https://www.tripadvisor.com/TravelersChoice-Destinations-cCulture-g1'
}

In [4]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

if TAVILY_API_KEY:
    print("Tavily API Key loaded successfully!")
else:
    print("Failed to load Tavily API Key. Please check your .env file.")


Tavily API Key loaded successfully!


In [5]:
# Step 1. Instantiating your TavilyClient
tavily_client = TavilyClient()

# Step 2. Executing a context search query
context = tavily_client.get_search_context(query="What happened during the Burning Man floods?")

# Step 3. That's it! You now have a context string that you can feed directly into your RAG Application
print(context)




"[\"{\\\"url\\\": \\\"https://www.aljazeera.com/news/2023/9/3/death-under-investigation-after-storm-flooding-at-burning-man-festival\\\", \\\"content\\\": \\\"just walked 5 miles in the mud out of burning man with chris rock and a fan picked us up pic.twitter.com/0uxSXLHgY6\\\\n\\\\u2014 diplo (@diplo) September 2, 2023\\\\nVehicle gates will not open for the remainder of the event, which began on August 27 and was scheduled to end on Monday, according to the US Bureau of Land Management, which oversees the Black Rock Desert where the festival is being held.\\\\n The Pershing County Sheriff\\\\u2019s Office said the death happened during the event in Black Rock City, but offered few details, declining to reveal the identity of the deceased person or the suspected cause of death, local TV channel KNSD reported.\\\\n Skip links\\\\nLive\\\\nNavigation menu\\\\nLive\\\\nDeath at Burning Man investigated in US, thousands stranded by flooding\\\\nThousands of festivalgoers remain stranded a

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

/var/folders/jx/75787np93351k9nr95h9tpp40000gn/T/ipykernel_6382/2234982945.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo")


In [14]:
# Define a Tavily crawling function
def tavily_crawl_and_extract(url: str) -> str:
    """
    Uses Tavily to crawl the URL and extract information from trip advisor and other websites.
    """
    query= "Top 5 desitinations in India"
    search = tavily_client.get_search_context(max_results=5, topic="travel", query = query)


    res = search.run(f"{query}")


In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("best travel desitnations from {topic}")

chain = prompt | llm | StrOutputParser() | (lambda output: output.replace("\n", ""))

chain.invoke('India')

"1. Maldives - Known for its crystal-clear waters, white sandy beaches, and luxury resorts, Maldives is a popular destination for Indian travelers looking for a relaxing beach vacation.2. Thailand - With its vibrant culture, delicious food, and stunning beaches, Thailand is a top choice for Indian travelers. From bustling cities like Bangkok to serene islands like Phuket, there is something for everyone in Thailand.3. Dubai - Dubai is a great destination for Indian travelers looking for a mix of luxury shopping, desert adventures, and modern architecture. The city is home to some of the world's tallest buildings, biggest malls, and most luxurious hotels.4. Switzerland - Known for its picturesque landscapes, charming villages, and outdoor activities, Switzerland is a popular destination for Indian travelers looking to experience the beauty of the Swiss Alps.5. Singapore - Singapore is a great destination for Indian travelers looking for a mix of culture, shopping, and delicious food. Th

In [17]:
def travel_search(query):
  for url in urls:
    return tavily_crawl_and_extract(url)

In [18]:
#Define tools for agent
travel_search = Tool(name = "TravelSearch",func= travel_search,description="Use this tool to get the travel information from tavily")
tools = [travel_search]

In [19]:
agent = initialize_agent(tools,llm,agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [21]:
#Define a query
query = "Please tell me travel destinations in and around India"

#Exexute agent
response = agent.run(query)

print(response)  # Inspect the data to find unexpected sets

#print("Agent response :",  response)



> Entering new AgentExecutor chain...
I should use the TravelSearch tool to get the travel information
Action: TravelSearch
Action Input: "travel destinations in and around India"

HTTPError: 400 Client Error: Bad Request for url: https://api.tavily.com/search